In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import cvxpy as cp
import scipy
from sklearn.linear_model import LassoLars
from helper import *
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
m = 10
D = 1 * np.diag(1*np.random.rand(m)) # np.eye(m)
print(np.max(D)/np.min(np.diag(D)))
Dhalf = np.sqrt(D)
rank = 1
L = 2 * np.random.randn(m,rank)
X = scipy.linalg.sqrtm(D + L @ L.T)
y = 10*np.random.randn(m) - 3
p = X.T @ y

21.033714671090152


In [66]:
## l0 brute force

if m <= 15:
    l0_costs, l0_supp = [], []
    l0_sols = []
    M_list = []
    for k in range(1,m+1):
        supp, cost = l0_bruteforce(D + L @ L.T, p, k)
        l0_costs.append(cost)
        l0_supp.append(np.where(np.array(supp) == 1)[0])
        
        #solution
        idx = np.where(np.array(supp) == 1)[0]
        Xtilde = X[:, idx]
        sol = np.linalg.inv(Xtilde.T @ Xtilde) @ Xtilde.T @ y
        l0_sols.append(sol)
        M_list.append(np.max(np.abs(sol)))

In [67]:
print(l0_sols[3])
print(l0_supp[3])

[-19.14082839  47.2820129   66.89453939 -28.43482328]
[4 7 8 9]


In [90]:
# simple dual ascent
niter = 10000
G = lambda w: X.T @ (X @ w - y)
xk, sk = np.zeros(m), np.zeros(m)
sum_x = np.zeros(m)
gk = G(xk)
v = np.zeros(m)
Sk = 0
lk = 0.1
gamma = 1
beta_list = [1]

for k in range(niter):
        
    # take gradient
    gk = G(xk)
    
    # update dual variable and running step size
    sk = sk + lk * gk
    Sk += lk
    
    # do projection onto l0 with l2-prox
    beta = beta_list[k] 
    v = -1/2 * (sk ** 2)/(gamma * beta)
    idx = np.argpartition(v, 4) #4 is sparsity level
    xk = np.zeros(m)
    xk[idx[:4]] = -sk[idx[:4]]/(gamma * beta)
    
    sum_x += lk * xk
    
#     if k % 1000 == 0:
#         print(f'Iter: {k}')
#         print(gk)
#         print(sk)
#         print(v)
#         print(xk)
#         print('----------------------------')

    
    # update beta
    beta_list.append(beta + 1/beta)

print(xk)


[  0.         -15.85429388   0.           0.           0.
   0.           0.          15.71774828  16.58586484 -22.31244328]


In [88]:
## dual solve

dual_costs, dual_supp = [], []
primal_feas_cost = []
for k in range(1,m+1):
    t, cost = cvx_dual(D, L, p, k)
    dual_idx = t.argsort()[-k:][::-1]
    dual_idx.sort()
    QQ = (D + L@ L.T)[dual_idx,:][:,dual_idx]
    pp = p[dual_idx]
    pf_cost = -1/2 * pp.T @ np.linalg.inv(QQ) @ pp
    dual_supp.append(dual_idx)
    dual_costs.append(-cost)
    primal_feas_cost.append(pf_cost)

In [91]:
print(l0_sols[3])
print(l0_supp[3])

[-19.14082839  47.2820129   66.89453939 -28.43482328]
[4 7 8 9]


In [17]:
xk

array([ 0.,  0.,  0.,  0., -0.,  0., -0., -0., -0.,  0.])

In [2]:
a = [1,2,5,1,4,40,500]

In [3]:
a

[1, 2, 5, 1, 4, 40, 500]

In [6]:
sorted(a)[:3]

[1, 1, 2]